In [1]:
!nvidia-smi

Sat Mar 19 06:52:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pkill -9 tmate
!wget -nc https://github.com/tmate-io/tmate/releases/download/2.4.0/tmate-2.4.0-static-linux-i386.tar.xz &> /dev/null
!tar --skip-old-files -xvf tmate-2.4.0-static-linux-i386.tar.xz &> /dev/null
!rm -f nohup.out; bash -ic 'nohup ./tmate-2.4.0-static-linux-i386/tmate -S /tmp/tmate.sock new-session -d & disown -a' >/dev/null 2>&1
!./tmate-2.4.0-static-linux-i386/tmate -S /tmp/tmate.sock wait tmate-ready
!./tmate-2.4.0-static-linux-i386/tmate -S /tmp/tmate.sock display -p "Connect with SSH address: #{tmate_ssh}"
!./tmate-2.4.0-static-linux-i386/tmate -S /tmp/tmate.sock display -p "Connect with web: #{tmate_web}"


Connect with SSH address: ssh mqGCW8XGjyFGnXvNWhNSg5VnF@nyc1.tmate.io
Connect with web: https://tmate.io/t/mqGCW8XGjyFGnXvNWhNSg5VnF


In [3]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(10000000):
		a[i]+= 1	

						
def func2(a):
	for i in range(10000000):
		a[i]+= 1
if __name__=="__main__":
	n = 10000000							
	a = np.ones(n, dtype = np.float64)
	b = np.ones(n, dtype = np.float32)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)
	

without GPU: 3.9536475290000084
with GPU: 3.9130389279999918


In [ ]:
!sudo apt install cpulimit
!top  

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cpulimit is already the newest version (2.5-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
=top - 07:29:46 up 38 min,  0 users,  load average: 0.02, 0.01, 0.00
Tasks:  14 total,   1 running,  12 sleeping,   0 stopped,   1 zombie
%Cpu(s):  2.1 us,  1.3 sy,  0.0 ni, 96.4 id,  0.2 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 13302920 total,  9762180 free,  1089184 used,  2451556 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 12007452 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND   
     72 root      20   0  731140 279688  68140 S   6.7  2.1   0:14.42 python3   
      1 root      20   0     992      4      0 S   0.0  0.0   0:00.46 docker-i+ 
      7 root      20   0  342372  52884  32680 S   0.0  0.4   0:01.59 node      
     18 root      20   0   35888   4828   3720 S   0.0  0.0   0:00.54 tail      
     28 root   

In [ ]:
!cpulimit -l 90 -b